In [ ]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *

---

Fine tuning on ordinary spherical kriging...

{'nlags': 8, 'variogram_parameters': {'psill': 1.5, 'range': 1.5, 'nugget': -1}, 'weight': False}

In [ ]:
from pykrige.rk import Krige

main_dir = '/nfs/home/genovese/thesis-wildfire-genovese/rekriging_target/'

In [ ]:
import datetime

def get_month_doy_range(month: int, year: int) -> tuple:
    """
    Given a month and a year, returns the first and last day of the month
    as day-of-year (DOY) values.
    """
    # First day of the month
    first_day = datetime.date(year, month, 1)
    # Handle December separately to avoid month rollover issues
    if month == 12:
        last_day = datetime.date(year, 12, 31)
    else:
        # First day of next month minus one day gives the last day of the current month
        last_day = datetime.date(year, month + 1, 1) - datetime.timedelta(days=1)

    return (first_day.timetuple().tm_yday, last_day.timetuple().tm_yday)

In [ ]:
file_dir = '/nfs/home/genovese/thesis-wildfire-genovese/rekriging_target/database/cache/by_season/'

fire_id = -1

for file in [f for f in os.listdir(file_dir) if 'negative' in f]:
    df = pd.read_csv(file_dir+f'{file}', index_col=0)
    # print(fire_id)
    # print(fire_id-df.shape[0])
    # print(list(range(fire_id, fire_id-df.shape[0], -1)))
    df['fire_id'] = list(range(fire_id, fire_id-df.shape[0], -1))
    df['target'] = [0]*df.shape[0]
    fire_id += -df.shape[0]
    df.to_csv(f'/nfs/home/genovese/thesis-wildfire-genovese/rekriging_target/database/cache/by_season_target_index_assigned/{file}')


In [ ]:
for file in [f for f in os.listdir(file_dir) if 'positive' in f]:
    df = pd.read_csv(file_dir+f'{file}', index_col=0)
    df['target'] = 1
    df.to_csv(f'/nfs/home/genovese/thesis-wildfire-genovese/rekriging_target/database/cache/by_season_target_index_assigned/{file}')

In 2017_winter_positive.csv, 2017_spring_positive.csv, 2021_winter_positive.crs I had to manually delete some row that had no point reference

In [ ]:
from itertools import product
import pickle
from shapely import wkt

for s, y in tqdm(product(['winter', 'spring', 'summer', 'autumn'], list(range(1997, 2025)))):
    p = pd.read_csv(main_dir+f'database/cache/by_season_target_index_assigned/{y}_{s}_positive.csv')
    if p.shape[0] == 0:
        continue
    p['geometry'] = p['geometry'].apply(wkt.loads)
    positives = gpd.GeoDataFrame(p, geometry='geometry')
    
    n = pd.read_csv(main_dir+f'database/cache/by_season_target_index_assigned/{y}_{s}_negative.csv')
    if n.shape[0] == 0:
        continue

    n['geometry'] = n['geometry'].apply(wkt.loads)
    negatives = gpd.GeoDataFrame(n, geometry='geometry')

    tot = pd.concat([positives, negatives])

    # Create Kriging model (scikit-learn wrapper)

    kriging_model = Krige(method='ordinary', variogram_model='spherical', variogram_parameters={'psill': 1.5, 'range': 1.5, 'nugget': -1}, nlags=8, weight=False)
    kriging_model.fit(np.array(pd.concat([tot.geometry.x, tot.geometry.y], axis=1)), tot.target)
    with open(main_dir+f'database/kriged_maps/{y}_{s}.pkl', 'wb') as f:
        pickle.dump(kriging_model, f)


---

---